# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [1]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

tuning_job_name = 'tensorflow-training-190809-0211'

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [2]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

16 training jobs have completed


In [3]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2019, 8, 9, 2, 27, 40, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'val_acc',
                                                 'Value': 0.8335999846458435},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2019, 8, 9, 3, 11, 53, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-west-2:453691756499:training-job/tensorflow-training-190809-0211-011-8631cb28',
 'TrainingJobName': 'tensorflow-training-190809-0211-011-8631cb28',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2019, 8, 9, 2, 29, 27, tzinfo=tzlocal()),
 'TunedHyperParameters': {'batch-size': '"32"',
                          'learning-rate': '0.00018105441613943323',
                          'momentum': '0.909726281471405',
                          'optimizer': '"adam"'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [4]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 16
{'lowest': 0.10130000114440918, 'highest': 0.8335999846458435}


,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,batch-size,learning-rate,momentum,optimizer
5,0.8336,2546.0,2019-08-09 03:11:53+00:00,tensorflow-training-190809-0211-011-8631cb28,Completed,2019-08-09 02:29:27+00:00,"""32""",0.000181,0.909726,"""adam"""
1,0.8334,2266.0,2019-08-09 03:21:03+00:00,tensorflow-training-190809-0211-015-3e0f9f40,Completed,2019-08-09 02:43:17+00:00,"""32""",0.000150,0.985885,"""adam"""
0,0.8090,730.0,2019-08-09 02:56:16+00:00,tensorflow-training-190809-0211-016-a06025db,Completed,2019-08-09 02:44:06+00:00,"""1024""",0.000231,0.940054,"""adam"""
9,0.8076,739.0,2019-08-09 02:26:09+00:00,tensorflow-training-190809-0211-007-21fc5950,Completed,2019-08-09 02:13:50+00:00,"""512""",0.000148,0.971761,"""adam"""
14,0.8016,752.0,2019-08-09 02:26:10+00:00,tensorflow-training-190809-0211-002-a372d76f,Completed,2019-08-09 02:13:38+00:00,"""512""",0.000158,0.953466,"""adam"""
4,0.7965,724.0,2019-08-09 02:41:48+00:00,tensorflow-training-190809-0211-012-0885beee,Completed,2019-08-09 02:29:44+00:00,"""512""",0.002060,0.911723,"""sgd"""
2,0.7926,715.0,2019-08-09 02:42:46+00:00,tensorflow-training-190809-0211-014-3ba86f63,Completed,2019-08-09 02:30:51+00:00,"""1024""",0.002376,0.959406,"""sgd"""
11,0.7893,712.0,2019-08-09 02:26:01+00:00,tensorflow-training-190809-0211-005-5679c846,Completed,2019-08-09 02:14:09+00:00,"""1024""",0.001196,0.936041,"""adam"""
12,0.7853,738.0,2019-08-09 02:26:09+00:00,tensorflow-training-190809-0211-004-9e1ee199,Completed,2019-08-09 02:13:51+00:00,"""512""",0.004770,0.915472,"""sgd"""
10,0.7811,1603.0,2019-08-09 02:40:33+00:00,tensorflow-training-190809-0211-006-08d4e93b,Completed,2019-08-09 02:13:50+00:00,"""32""",0.001302,0.926853,"""sgd"""


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [12]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For exmaple, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [13]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))